In [1]:
import pandas as pd
import numpy as np

In [2]:
df_bm_train = pd.read_csv("bigmart_train.csv")
df_bm_test = pd.read_csv("bigmart_test.csv")

In [3]:
df_median_item_weights = df_bm_train.groupby(["Item_Type"],as_index = False).agg({"Item_Weight":["median"]})

In [4]:
df_median_item_weights.columns = ["Item_Type","median_Item_Weight"]

In [5]:
df_bm_train = pd.merge(left = df_bm_train,right = df_median_item_weights,on = ["Item_Type"],how ="left")

In [6]:
df_bm_train['Item_Weight'] = df_bm_train.apply(lambda x : x['median_Item_Weight'] if pd.isna(x['Item_Weight']) else x['Item_Weight'],axis = 1)

In [7]:
df_bm_train["Outlet_Size"] = df_bm_train["Outlet_Size"].apply(lambda x : "Medium" if pd.isna(x) else x)

In [8]:
df_bm_train["Item_Fat_Content"] = df_bm_train["Item_Fat_Content"].map({'Low Fat':"Low Fat",'Regular':"Regular",'low fat':"Low Fat",'LF':"Low Fat",'reg':"Regular"})

In [9]:
df_bm_train.drop(["Item_Identifier","Outlet_Identifier","median_Item_Weight"],axis =1 , inplace = True)

In [10]:
df_bm_train1 = pd.get_dummies(columns = ["Item_Fat_Content","Item_Type","Outlet_Size","Outlet_Location_Type","Outlet_Type"]
                                         ,drop_first = True , data = df_bm_train)

In [11]:
df_bm_train1["Outlet_Age"]= df_bm_train1["Outlet_Establishment_Year"].apply(lambda x : 2022 - x)

In [12]:
df_bm_train1.drop(columns = 'Outlet_Establishment_Year',inplace = True)

In [13]:
x_train = df_bm_train1.drop(["Item_Outlet_Sales"],axis = 1)

In [14]:
y = df_bm_train1[["Item_Outlet_Sales"]]

In [15]:
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
x = scaler.fit_transform(x_train)

In [16]:
from sklearn.model_selection import train_test_split as tts
xtr,xtt,ytr,ytt = tts(x,y,test_size=0.3,random_state = 123)

In [17]:
import pyforest
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix as smx

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.metrics import mean_squared_error as mse

import  warnings
warnings.filterwarnings('ignore')

In [ ]:
lazy = LazyRegressor(ignore_warnings=False , custom_metric=None)
models,predictions =  lazy.fit(xtr,xtt,ytr,ytt)

 74%|████████████████████████████████████████████████████████████▌                     | 31/42 [00:52<00:15,  1.43s/it]